In [1]:
import os
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
#sns.set(style="darkgrid")
from keras.models import load_model

Using TensorFlow backend.


In [2]:
df = pd.read_excel("time_series_all.xlsx")

In [3]:
df 

,Feeder,Measurand,0,1,2,3,4,5,6,7,...,14490,14491,14492,14493,14494,14495,14496,14497,14498,14499
0,0,KV,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B-4 S.J. ENCL.,AMPs,59.279,59.279,59.279,59.279,55.2146,55.2146,55.2146,55.2146,...,40.7447,40.7447,40.7447,50.2752,50.2752,50.2752,50.2752,62.5414,62.5414,62.5414
2,B-4 S.J. ENCL.,PF,-0.981392,-0.981392,-0.981392,-0.981392,-0.981995,-0.981995,-0.981995,-0.981995,...,-0.964221,-0.964221,-0.964221,-0.973871,-0.973871,-0.973871,-0.973871,-0.98305,-0.98305,-0.98305
3,PANKHA RD CKT-2,AMPs,84.2096,84.2096,84.2096,84.2096,71.1225,71.1225,71.1225,71.1225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PANKHA RD CKT-2,KV,68.7123,68.7123,68.7123,68.7123,68.7763,68.7763,68.7763,68.7763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PANKHA RD CKT-2,PF,-0.993832,-0.993832,-0.993832,-0.993832,-0.994503,-0.994503,-0.994503,-0.994503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,IGNOU S/S -3(Feeder 1),AMPs,50.6842,50.6842,50.6842,50.6842,47.0453,47.0453,47.0453,47.0453,...,33.5321,33.5321,33.5321,38.4531,38.4531,38.4531,38.4531,47.1678,47.1678,47.1678
7,IGNOU S/S -3(Feeder 1),PF,-0.996355,-0.996355,-0.996355,-0.996355,-0.996485,-0.996485,-0.996485,-0.996485,...,-0.997994,-0.997994,-0.997994,-0.998952,-0.998952,-0.998952,-0.998952,-0.998727,-0.998727,-0.998727
8,1-OKHLA (SPARE),AMPs,21.7221,21.7221,21.7221,21.7221,19.3694,19.3694,19.3694,19.3694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1-OKHLA (SPARE),KV,66.5628,66.5628,66.5628,66.5628,66.762,66.762,66.762,66.762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna()

In [5]:
df = df.reset_index(drop=True)

In [20]:
l = list(range(0,14000,4))
l.insert(0,'Feeder')
l.insert(1,'Measurand')


## Resolution decreased - 1 hr

In [22]:
df = df[l]

In [ ]:
def isSubArray(A, B, n, m, st):
    ii = st
    j = 0
    while (ii < n and j < m): 
        if (A[ii] == B[j]): 
            ii += 1 
            j += 1 
            if (j == m): 
                return ii,True
        else: 
            ii = ii - j + 1 
            j = 0 
    return ii,False 

In [ ]:
l = []
for i in df.index:
    xx = df.iloc[i][:250].values
    r = isSubArray(xx.tolist(),[0,0,0,0], 250, 4,0)
    if(r[1]):
        l.append((i, r[0]))                      # i=index, r[0]='timestamp', r[1]=fault
l = pd.DataFrame(l)

In [ ]:
l

In [ ]:
df = df.drop(l[0])

In [ ]:
df

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

# faults

In [ ]:
l = []
for i in df.index:
    xx = df.iloc[i][250:].values
    r = isSubArray(xx.tolist(),[0,0,0,0], len(xx), 4,0)
    if(r[1]):
        l.append((i, r[0]))
l = pd.DataFrame(l)

In [ ]:
# l: 0=index, 1='timestamp', 2=fault
X_fault_indices = pd.DataFrame(zip(l[0], l[1], l[1].apply(lambda t: list(range(t-4, t+250-4))))) # l[1] is already fault-250
X_fault = []
for i in X_fault_indices.values:
    X_fault.append([i[0], list(df.iloc[i[0], i[2]])])
    
X_fault = pd.DataFrame(X_fault)                 # 0 - df index for future use
X_fault = pd.DataFrame(list(X_fault[1].values))

y is just an list of ones. test_train_split() will divide properly

# no fault
selected randomly from !l

In [ ]:
mask = pd.Series(len(df) * [True])
mask[l[0]] = False
no_faults = df[mask]

In [ ]:
X_no_faults = []

for i in no_faults.index:
    r = np.random.randint(df.shape[1] - 250)
    X_no_faults.append([i, list(df.iloc[i,r:r+250])])

X_no_faults = pd.DataFrame(list( pd.DataFrame(X_no_faults)[1].values ))

In [ ]:
for i in range(3):
    X_no_faults = pd.concat([X_no_faults, X_no_faults])
    X_fault = pd.concat([X_fault, X_fault])

y = np.array([1]*len(X_fault) + [0]*len(X_no_faults))

X = np.array(pd.concat([X_fault, X_no_faults]))

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Spliting into traing and testing 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
x_train.shape,x_test.shape

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(256, input_shape=(x_train.shape[1],1))) # dropout = 0.2,recurrent_dropout=0.2,

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy' , optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs=50,validation_split=0.2,batch_size=3000)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('time')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()


In [ ]:
pred = model.predict(x_train)
predict_classes = pred.argmax(axis = 1)
# Accuracy
print(sum(predict_classes==y_train)/len(y_train))

cm = confusion_matrix(y_train,predict_classes)
sns.heatmap(confusion_matrix(y_train,predict_classes),annot=True,fmt='d',cmap="Blues") 
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
pred = model.predict(x_test)
predict_classes = pred.argmax(axis = 1)
# Accuracy
print(sum(predict_classes==y_test)/len(y_test))

cm = confusion_matrix(y_test,predict_classes)
sns.heatmap(confusion_matrix(y_test,predict_classes),annot=True,fmt='d',cmap="Blues") 
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
pred